# National Tsing Hua University Robust and Stochastic Portfolio Optimization (Fall 2021)

 - Week 3 Programming HW

In [2]:
# basic
import numpy as np 
import pandas as pd
import datetime as dt 

# yahoo data source
import yfinance as yf 
from pandas_datareader import  data as pdr 

# optimization package
import cvxpy as cp 
from scipy.optimize import minimize
from scipy.spatial import ConvexHull, convex_hull_plot_2d

# matplotlib
import matplotlib.pyplot as plt 
plt.style.use('ggplot')

In [3]:
start_date = '2019-01-02'
end_date   = '2020-01-02'

# yfinance likes the tickers formatted as a list
ticks    = yf.Tickers(["FB","MSFT","AAPL","AMZN","NFLX",'GOOG','TSLA','AMD','^DJI','JPM'])
Stock_DF = ticks.history(start=start_date, end=end_date).Close
Stock_DF = Stock_DF.fillna(value=0)

Stock_DF

[*********************100%***********************]  10 of 10 completed


,AAPL,AMD,AMZN,FB,GOOG,JPM,MSFT,NFLX,TSLA,^DJI
Date,,,,,,,,,,
2019-01-02,38.326290,18.830000,1539.130005,135.679993,1045.849976,90.652054,97.961327,267.660004,62.023998,23346.240234
2019-01-03,34.508709,17.049999,1500.280029,131.740005,1016.059998,89.363747,94.357521,271.200012,60.071999,22686.220703
2019-01-04,35.981865,19.000000,1575.390015,137.949997,1070.709961,92.658180,98.746017,297.570007,63.537998,23433.160156
2019-01-07,35.901775,20.570000,1629.510010,138.050003,1068.390015,92.722588,98.871948,315.339996,66.991997,23531.349609
2019-01-08,36.586166,20.750000,1656.579956,142.529999,1076.280029,92.547737,99.588860,320.269989,67.070000,23787.449219
...,...,...,...,...,...,...,...,...,...,...
2019-12-24,70.027313,46.540001,1789.209961,205.119995,1343.560059,129.506668,154.718262,333.200012,85.050003,28515.449219
2019-12-26,71.416679,46.630001,1868.770020,207.789993,1360.400024,130.880981,155.986450,332.630005,86.188004,28621.390625
2019-12-27,71.389572,46.180000,1869.800049,208.100006,1351.890015,130.975143,156.271561,329.089996,86.075996,28645.259766


In [22]:
Expected_Return   = Stock_DF.pct_change().mean() 
Unbiased_Std      = Stock_DF.pct_change().std()


CovexHull_DF = pd.DataFrame({
    "Unbiased_Std" : Unbiased_Std ,
    "Expected Returns" : Expected_Return ,
    })


print("-" * 40)
print(CovexHull_DF)
print("-" * 40)

----------------------------------------
      Unbiased_Std  Expected Returns
AAPL      0.016498          0.002671
AMD       0.033971          0.004111
AMZN      0.014382          0.000832
FB        0.017471          0.001801
GOOG      0.015228          0.001094
JPM       0.011775          0.001543
MSFT      0.012509          0.001909
NFLX      0.021853          0.000994
TSLA      0.030817          0.001668
^DJI      0.007850          0.000831
----------------------------------------


# Problem 2.6 ( Portfolio Opimization Via MAD Objective )

$$ Max f(x) =  E | W^T (r-u)| $$

s.t. $$ w \in w := { w : \sum_{i=1}^{n}|w_i| = 1 } $$

$$  \forall_{i} = 1,2,.....,10  $$ 


Then For Each day k =1,.....,N , we assume that the probability for obtaining the daily return 


$$ vector r(k) = [r_1(k) , r_2(k) ,.....,r_n(k)]^T \in R^n $$



In [29]:
class CVX_MAD_Optimization():
    
    
    def __init__(self,u,daily_r):

        self.u         = u
        self.daily_r   = daily_r
        self.params    = cp.Variable(u.shape[0])

    def constraint_1(self):
        
        return cp.sum(self.params) == 1 # Weights Sum 1 
    

    def constraint_2(self):

        bound = [ self.params[i] >= float(1e-20) for i in range(self.u.shape[0]) ] 

        return bound 

    def Optimize(self):

        constraints = []
        constraints.extend( [self.constraint_1()] )
        constraints.extend(  self.constraint_2()  )
        
        Objective_Function     =   ( np.abs( self.daily_r - self.u/252 ) ).sum(axis=0) @ self.params  * (1/self.u.shape[0])
        # Objective_Function   =   -cp.sum( ( cp.abs( self.daily_r -self.u  ) @ self.params )  )  * (1/self.u.shape[0])


        prob  = cp.Problem( cp.Minimize(Objective_Function) , constraints )
        
        return prob , self.params


Cov_Matrix = Stock_DF.pct_change().cov() * np.sqrt(252)
u          = Expected_Return.values  * 252 
sigma      = Unbiased_Std.values * np.sqrt(252)

In [30]:
Model  = CVX_MAD_Optimization( daily_r=Stock_DF.pct_change().dropna().values , u=u )
Result,Params = Model.Optimize()
print("Min F(x) : "       , Result.solve()              )
print("Weights Values : " , np.round(Params.value,5)    )
print("Weights Sum    : " , np.sum(Params.value)        )

Min F(x) :  0.13849811396269793
Weights Values :  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Weights Sum    :  1.0000000000000029


In [31]:
class CVX_Markowitz_Optimization():
    

    def __init__(self,u,cov):

        self.u      = u
        self.cov    = cov 
        self.params = cp.Variable(cov.shape[1])

    def constraint_1(self):
        
        return cp.sum(self.params) == 1 # Weights Sum 1 

    def constraint_2(self):

        bound = [ self.params[i] >= float(1e-20) for i in range(self.cov.shape[1]) ] 

        return bound 

    def Optimize(self):

        constraints = []
        constraints.extend( [self.constraint_1()] )
        constraints.extend(  self.constraint_2() )
        
        Objective_Function = self.u.T @ self.params  - (1/2) *  cp.quad_form(self.params ,self.cov) 
        # Objective_Function = (1/2) *  cp.quad_form(self.params ,self.cov) <-- Minimize Variance

        prob  = cp.Problem( cp.Maximize(Objective_Function) , constraints )
        
        return prob , self.params


Cov_Matrix = Stock_DF.pct_change().cov() * np.sqrt(252)
u          = Expected_Return.values * 252
sigma      = Unbiased_Std.values  * np.sqrt(252)

In [32]:
Model  = CVX_Markowitz_Optimization(cov=Cov_Matrix ,u=u )
Result,Params = Model.Optimize()
print("Max F(x) : "       , Result.solve()                      )
print("Weights Values : " , np.round(Params.value,decimals=5 )  )
print("Weights Sum    : " , np.sum(Params.value)                )

Max F(x) :  1.0266946437382836
Weights Values :  [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
Weights Sum    :  1.0
